In [33]:
import pandas as pd
df = pd.read_csv("RaceTimes.csv")

In [34]:
import pandas as pd
import re
from datetime import datetime

def convert_to_seconds(time_str):
    minutes, seconds, milliseconds = [int(part) for part in time_str.replace('.', ':').split(':')]
    return minutes * 60 + seconds + milliseconds / 1000

def convert_id_readable(race_id):
    # Use regular expressions to extract the time and date parts
    match = re.match(r"(\d+):(\d+)(\d{2})\.(\d{2}\.\d{4})", race_id)
    
    if not match:
        return "Invalid format"

    # Extract the hour, minute, day, and the rest of the date parts
    hour, minute, day, month_year = match.groups()
    
    # Format the hour to 12-hour time and assume it is PM
    hour_int = int(hour)
    # If the hour is less than 12, we assume it's PM, otherwise, we convert to 12-hour format
    hour_formatted = f"{hour_int if hour_int <= 12 else hour_int - 12}"
    time_suffix = "pm" if hour_int <= 12 else "am"
    
    # Construct the time with the correct suffix
    time = f"{hour_formatted}:{minute}{time_suffix}"
    
    # Construct the date in the desired format
    date_formatted = datetime.strptime(f"{day}.{month_year}", "%d.%m.%Y").strftime("%d/%m/%Y")
    
    # Return the formatted string
    return f"{time} @ {date_formatted}"

df['Lap Time Seconds'] = df['Lap Time'].apply(convert_to_seconds)
df['RaceID Name'] = df['RaceID'].apply(convert_id_readable)
print(df)


    Time of Race        Date             Racer  Lap   Lap Time  \
0           4:52  01.10.2023  josh.kolappillil    1  01:36.378   
1           4:52  01.10.2023  josh.kolappillil    2  00:42.928   
2           4:52  01.10.2023  josh.kolappillil    3  00:41.775   
3           4:52  01.10.2023  josh.kolappillil    4  00:42.382   
4           4:52  01.10.2023  josh.kolappillil    5  00:45.255   
..           ...         ...               ...  ...        ...   
345         5:32  06.01.2024       darian.king   10  00:37.822   
346         5:32  06.01.2024       darian.king   11  00:38.199   
347         5:32  06.01.2024       darian.king   12  00:38.118   
348         5:32  06.01.2024       darian.king   13  00:37.781   
349         5:32  06.01.2024       darian.king   14  00:39.492   

             RaceID  Lap Time Seconds          RaceID Name Race Time  \
0    4:5201.10.2023            96.378  4:52pm @ 01/10/2023    4:52pm   
1    4:5201.10.2023            42.928  4:52pm @ 01/10/2023    4

In [35]:
# Correctly split 'RaceID Name' into 'Race Date' and 'Race Time'
split_data = df['RaceID Name'].str.split('@ ', n=1, expand=True)
df['Race Time'] = split_data[0]
df['Race Date'] = split_data[1]

print(df)


    Time of Race        Date             Racer  Lap   Lap Time  \
0           4:52  01.10.2023  josh.kolappillil    1  01:36.378   
1           4:52  01.10.2023  josh.kolappillil    2  00:42.928   
2           4:52  01.10.2023  josh.kolappillil    3  00:41.775   
3           4:52  01.10.2023  josh.kolappillil    4  00:42.382   
4           4:52  01.10.2023  josh.kolappillil    5  00:45.255   
..           ...         ...               ...  ...        ...   
345         5:32  06.01.2024       darian.king   10  00:37.822   
346         5:32  06.01.2024       darian.king   11  00:38.199   
347         5:32  06.01.2024       darian.king   12  00:38.118   
348         5:32  06.01.2024       darian.king   13  00:37.781   
349         5:32  06.01.2024       darian.king   14  00:39.492   

             RaceID  Lap Time Seconds          RaceID Name Race Time  \
0    4:5201.10.2023            96.378  4:52pm @ 01/10/2023   4:52pm    
1    4:5201.10.2023            42.928  4:52pm @ 01/10/2023   4:

In [40]:
df = df.sort_values(by=['Race Date', 'Race Time'], ascending=False)

df['Race Date'] = pd.to_datetime(df['Race Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Sort by 'Race Date' then 'Race Time'
df = df.sort_values(by=['Race Date', 'Race Time'], ascending=False)

df.to_csv('test.csv', index=False)